<h1>Processando a informação da tabela<br>Declarações de bilios e cia</h1>

In [ ]:
import pandas as pd
# tirar os avisos de quando 'mudar' o dataframe, para saber mais 
# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
pd.set_option("mode.chained_assignment", None)
import matplotlib.pyplot as plt
from collections import Counter
from babel.numbers import format_currency
import json
from datetime import date
import csv
import chardet
import datetime
import openpyxl

# importando FROM THA HOUSE
import horario
import dataframe_customizado as df_custom

## Criando a variavel com o caminho do CSV

In [ ]:
path = "" # caminho que está nosso csv do pagar.me

# MÉTODOS/CLASSES

In [ ]:
import pandas as pd
class DataframeClasse:
    def __init__(self,path):
        self.path = path
        self._df = self.criar_df()
        self.set_datas_datetime()
        
    # agora que temos o caminho para os dados, criamos o dataframe
    def criar_df(self):
        """Durante a instancia, faz a criação do dataframe."""
        if ".csv" in self.path:
            with open(self.path,'rb') as file:
                codificacao = chardet.detect(file.read())
                tipo_codi = codificacao['encoding']
            try:
                self._df = pd.read_csv(self.path,encoding = tipo_codi)
            except BaseException as err:
                print(err)
            return self._df
        else:
            print("Só suportamos arquivos .csv por enquanto")
    
    def get_df(self):
        return self._df
    
    def set_df(self,df):
        self._df = df
        
    def set_datas_datetime(self):
        """
        Transformar as datas do dataframe no formato datetime.
        """
        for cols in self._df.columns:
            cond_cols = cols.upper()
            if 'DATA' in cond_cols or 'DATE' in cond_cols:
                try:
                    self._df[cols] = pd.to_datetime(self._df[cols], format="%Y-%m-%d", errors='ignore')
                except BaseException as err:
                    print("Checa se não existe nenhuma coluna que não é formato de data, mas em seu nome tem 'DATA' ou 'DATE'")
                    raise Exception(err)
    
    # gerando um dataframe com os valores como parametro
    def get_df_valores(self,*valores):
        """
        Cria um df somente com os valores que coloca como parametro.
        Pode ser passado tanto como inteiros separados com virgula, como: get_df_valores(26, 8, 87)
        Ou dentro de uma lista, ex: get_df_valores([10,13,80]).
        """
        if len(valores) == 0:
            raise Exception(
                    """Não esqueça de passar os valores como parâmetro. Ex: <ObjetoClasse>.get_df_valores(valores)
                     Rode a expressão '<ObjetoClasse>.get_df_valores.__doc__' para ver a documentacao do método.
                    """)
        
        _lista_strings = []
        _string_filter = ''

        # pegando os valores no parametro e criando uma string com eles
        for val in valores:
            # checa se não foi passado em uma lista, se for pega a primeira
            if type(valores[0]) == list:
                for list_valores in val:
                    _lista_strings.append(f"(self._df['valor_brl'] == {list_valores})")
            # se nao for lista, roda o loop normal com os valores
            else:
                _lista_strings.append(f"(self._df['valor_brl'] == {val})")

        # juntando numa string, é os parametros para o filtro. | => or
        _string_filter = " | ".join(_lista_strings)
        # agora juntamos na expressão que vamos usar
        _string_resultado = f"self._df[{_string_filter}]"

        # roda a string como uma expressão lida como código
        return eval(_string_resultado)
    

    
    # métodos não essenciais
    def get_colunas(self,opcao = None):
        """
        Devolve as colunas do dataframe, se quiser mostra tbm o indice
        de cada com seu nome. 'S' para Sim e 'N' para Não
        """
        
        # se n quiser, só retorna as colunas do dataframe
        _cond2 = str(opcao).upper().startswith("N")
        if None or _cond2:
            return self._df.columns
        
        elif "oi" in opcao:
            print("olá")
            
        # se voce quiser, printa as colunas do dataframe tbm
        else:
            print("     Indice | Nome Coluna")
            print("------------------------------")
            for n,cols in enumerate(self._df.columns):
                print("| {:2} |  {:20} |".format(n,cols))
            print("------------------------------")
            return self._df.columns
    
    # Datas
    def get_intervalo_datas(self):
        _data_MAX = max(self._df['data_criacao'])
        _data_MIN = min(self._df['data_criacao'])
        print("Diferença de: {}".format((_data_MAX - _data_MIN)))
        print("Entre o dia {} e o dia {}".format(_data_MIN.date(),_data_MAX.date()))
        return _data_MAX - _data_MIN


## Classe para o dataframe do EadBox

In [ ]:
import pandas as pd
class DataFrame_eadbox(DataframeClasse):
    def __init__(self,path):
        super().__init__(path)
        self.set_colunas_eadbox()
        
    """ 
    When started, this is instantiated:   
        self.path = path
        self._df = self.criar_df()
        self.set_datas_datetime()
        
        get_df() -> returning the <instance>.dataframe
        set_df() -> setting the new df to <instance>.dataframe
    """

    
    def set_colunas_eadbox(self):
        list_col_original = [_ for _ in self.get_df().columns]
        list_col_changed = ['nome','email','cidade','estado','produto_tipo',
                            'produto_titulo','campanha_titulo','metodo_pag',
                            'cupom','status_detalhado','info_extra','status',
                            'valor_brl','data_criacao','data_atualizacao']
        dic_cols_mudar = {ori:chan for (ori,chan) in zip(list_col_original,list_col_changed)}
        _cond = self.get_df().rename(columns=dic_cols_mudar)
        self.set_df(_cond)
    

# Dataframe Pagar.me

In [ ]:
# instanciando a classe e checando o dataframe
obj_df = DataframeClasse(path)

# passamos o dataframe do obj para a variavel df, não é uma copia dela, referencia a propria do objeto
df = obj_df.get_df()
df_copy = df.copy()

# observamos as colunas do dataframe
obj_df.get_colunas("s")
print()

# retornar um dataframe com o status como PAGO, e sem algum valor estornado
df_status_pago = df[(df['status'] == 'paid') & (df['valor_estornado_brl'] <= 0)]

# ordenar o dataframe pela data de atualização
df_status_pago = df_status_pago.sort_values(by=['data_atualizacao'], ascending=False)

# objetivo é devolver o obj com nosso dataframe e ja puxando um com transacoes realmente pagas/ativas

## Vamos "Splitar" o dataframe em dois, com os assinantes e outro com planos => sendo considerado o 'valor_brl' para divisao

In [ ]:
# deixando a data em formato datetime64 ns, ao inves de ns UTC
df_status_pago['data_atualizacao'] = pd.to_datetime(df_status_pago.data_atualizacao).dt.tz_localize(None)

# tirando as duplicadas do df_status_pago
df_pago_unico = df_status_pago.drop_duplicates(subset=['email','valor_brl'], keep='first')

print("Emails duplicados: {}".format(sum([_ for _ in df_pago_unico['email'].value_counts() if _ > 1])))

# desde qual dia é para puxar?
print("Desde que dia é para filtrar?")
ano, mes, dia = horario.get_data_separada()

# criando um dataframe com valores unicos usando data como parametro
df_pagamentos_unicos = df_pago_unico[(df_pago_unico['data_atualizacao'] > (pd.Timestamp(year=ano,month=mes,day=dia,
                                                                                        hour=0).to_datetime64()))]

print()
print("Pagantes únicos desde o dia {:02d}/{:02d}/{}".format(dia,mes,ano))
print(df_pagamentos_unicos['valor_brl'].value_counts())

# objetivo, fazer analise por periodo (checar), mudar a data para conseguirmos tratar e criacao do df com pag únicos.

In [ ]:
# pegando os valore que possuem subscription id
lista_valores_subs = [100]
for valor,qtde in df_status_pago[df_status_pago['subscription_id'] != 'None']['valor_brl'].value_counts().items():
    print("R$ {:,} | {}".format(valor,qtde))
    
print()
# tirando o subscription_id do 1992
df_status_pago.loc[df_status_pago['valor_brl'] == 1992, ['subscription_id']] = "None"
print("Depois de deixar o valor de 1992.0 sem nenhum subscription_id")
for valor,qtde in df_status_pago[df_status_pago['subscription_id'] != 'None']['valor_brl'].value_counts().items():
    lista_valores_subs.append(valor)
    print("R$ {:,} | {}".format(valor,qtde))
    
print()

lista_valores_resto = []
for valores,i in df['valor_brl'].value_counts().items():
    if valores not in lista_valores_subs:
        lista_valores_resto.append(valores)
        
print("lista com valores da assinatura: {}".format(lista_valores_subs))
print("lista com o resto: {}".format(lista_valores_resto))
# obj, conseguimos a lista com os valores que vamos usar para dividir o df

### celula que fazemos a separação

In [ ]:
# passar os valores que possuem o subscription_id

# passamos o dataframe com valores pago como atributo
obj_df.set_df(df_status_pago)
print("Tamanho df_status_pago: {}".format(len(obj_df.get_df())))
df_subscribers = obj_df.get_df_valores(lista_valores_subs)
df_plans = obj_df.get_df_valores(lista_valores_resto)

print("{}\n{:{}} +\n---\n{}\n".format(len(df_subscribers),len(df_plans), len(str(len(df_plans)))+1, len(df_subscribers) + len(df_plans)))
print(df_subscribers['valor_brl'].value_counts())
print(df_plans['valor_brl'].value_counts())

# df_subscribers

In [ ]:
# As are recurring payments, we want the active students
# Who made a payment on the last 30 days, so its a active student.
# So we make a filter on dataframe selecting the last 30 days (1 month)

df_subscribers.sort_values(by=['data_atualizacao'], ascending=False, inplace=True)
df_subscribers.drop_duplicates(subset=['email','valor_brl'],inplace=True)

# creating variables to filter the dataframe, default = 1 month (30 days)
period_filter = datetime.datetime.now() - datetime.timedelta(days=30)

_year_filter = period_filter.year
_month_filter = period_filter.month
_day_filter = period_filter.day

# now make the filter
df_subscribe_filtered = df_subscribers[(df_subscribers['data_atualizacao'] > (pd.Timestamp(
                                                        year=_year_filter,month=_month_filter,
                                                        day=_day_filter,hour=0).to_datetime64()))]
                                                        # end the expression of filter
# --

df_subs_filt_active = df_subscribe_filtered.filter(items=['id','valor_brl']).groupby(by='valor_brl').count()
df_subs_filt_active.rename(columns={"id":"Alunos Ativos"},inplace=True)
df_subs_filt_active['Faturamento'] = df_subs_filt_active.index * df_subs_filt_active['Alunos Ativos']
print("Total: R$ {}".format(df_subs_filt_active['Faturamento'].sum()))
df_subs_filt_active

In [ ]:
# pegando os valores por dia

df_fil_subs = df_subscribe_filtered.filter(items=['valor_brl','data_atualizacao'])
df_subs_value = df_fil_subs['valor_brl']
df_subs_date = df_fil_subs['data_atualizacao']


l_valor = list()
l_date = list()
dic_value_date_subs = dict()

c = 0
for v,d in zip(df_subs_value,df_subs_date):
    d = str(d.date())
    #l_date.append(d)
    #l_valor.append(v)
    
    if d not in dic_value_date_subs:
        dic_value_date_subs[d] = 0
    dic_value_date_subs[d] += v

# print(l_date)
# print(l_valor)
dic_value_date_subs

## df_plans 

In [ ]:
df_plans.groupby(by='valor_brl').count()

df_plans_active = df_plans[df_plans['subscription_id'] == "None"].filter(items=['id','valor_brl']).groupby(by='valor_brl').count()

In [ ]:
df_plans_date = df_custom.get_date_filter(df_plans,"2022-03-01","2022-03-14")

df_plans_filter = df_plans_date.filter(items=['valor_brl','valor_captura_brl']).groupby(by="valor_brl").count()
df_plans_filter['Faturado'] = df_plans_filter.index * df_plans_filter['valor_captura_brl']
print("Total: R$ {}".format(df_plans_filter['Faturado'].sum()))
df_plans_filter

In [ ]:
# pegando os valores por dia

df_fil_plans = df_plans_date.filter(items=['valor_brl','data_atualizacao'])
df_plans_value = df_fil_plans['valor_brl']
df_plans_date = df_fil_plans['data_atualizacao']


l_valor = list()
l_date = list()
dic_value_date_plans = dict()

c = 0
for v,d in zip(df_plans_value,df_plans_date):
    d = str(d.date())
    
    if d not in dic_value_date_plans:
        dic_value_date_plans[d] = 0
    dic_value_date_plans[d] += v

dic_value_date_plans

In [ ]:
check = df_custom.get_date_filter(df_status_pago,"2022-02-10","2022-03-10")
print(f"Data Maxima: {check['data_atualizacao'].max()}")
print(f"Data Minima: {check['data_atualizacao'].min()}")

check['valor_brl'].value_counts()

## Criando lista de compradores e checando com ultimos pagamentos

In [ ]:
df_ate_ontem = df_custom.get_date_filter(df_status_pago, "2021-10-20","2022-03-15")

lista_compradores_ontem = df_custom.get_lista_email(df_ate_ontem)


df_ate_hoje = df_custom.get_date_filter(df_status_pago,"2022-03-15")

print("Até ontem: {}".format(len(df_ate_ontem)))
print("Status Pago: {}".format(len(df_status_pago)))
print("Lista compradores: {}".format(len(lista_compradores_ontem)))

In [ ]:
df_ate_hoje['valor_brl'].value_counts()
c = 0
l = []
l_novo = []
for email_hoje in df_ate_hoje['email']:
    if email_hoje in lista_compradores_ontem:
        print("Email => {}, já estava na lista de compradores".format(email_hoje))
        l.append(email_hoje)
        c += 1
    else:
        novo_valor = df_ate_hoje[df_ate_hoje['email']==email_hoje]['valor_brl']
        l_novo.append(novo_valor)
print()        
print(f"Dos {len(df_ate_hoje['email'])} pagamentos de hoje para ontem, apenas {c} já eram pagantes")
print()
print("Dos novos pagantes, são esses valores")

#print(l_novo)
l_nn = []
for i in l_novo:
    l_nn.append(i.values.item())
    
contg_pagamentos = Counter(l_nn)
for v,q in contg_pagamentos.items():
    print(f"No valor de R$ {v} foram {q} novos pagantes")

In [ ]:
df_status_pago.sort_values(by='data_atualizacao', ascending=False)

## COMPRADORES COM PAGAMENTO DUPLO, (EMAIL IGUAL | VALOR DIFERENTE)

In [ ]:
d_t = df_pago_unico[df_pago_unico.duplicated(subset='email', keep=False)].sort_values(by=['email','data_atualizacao'])
#pd.set_option('display.max_rows', None)
d_t['valor_brl'].value_counts()
d_first = d_t.drop_duplicates(subset=['email'],keep='first')
d_last = d_t.drop_duplicates(subset=['email'],keep='last')
d_first.head()
print(f"Do dia {d_t['data_atualizacao'].min().date()} ate {d_t['data_atualizacao'].max().date()}")
print("Compradores com ultima ação sendo COMPRA, que fizeram +1 pedido com diferentes valores e mesmo email")
print("{} compradores".format(len(d_first)))
print()
print("Os valores da primeira compra são:")
print("Qtde -> valor")
for v,q in d_first['valor_brl'].value_counts().items():
    print("{:2} -> R$ {:,}".format(q,v))

print()
print("Quantia de valores ref a segunda compra")
print("Qtde -> valor")
for v,q in d_last['valor_brl'].value_counts().items():
    print("{:2} -> R$ {:,}".format(q,v))
    
c = 0
temp_medio = 0
for tempo_f,tempo_l in zip(d_first['data_atualizacao'],d_last['data_atualizacao']):
    #if tempo_l - tempo_f > datetime.timedelta(seconds=1800):
    temp_medio += (tempo_l-tempo_f).total_seconds()
        #print(temp_medio / 3600)
        #c += 1

print()
print("Com um tempo médio da primeira compra para a segunda de aproximadamente {}".format(datetime.timedelta(seconds= (temp_medio / len(d_last)))))
print(d_t['data_atualizacao'].min().date())
# mudando o tipo das datas para exportar como .xlsx
d_t['data_criacao'] = pd.to_datetime(d_t.data_criacao).dt.tz_localize(None)
d_t['data_atualizacao'] = pd.to_datetime(d_t.data_atualizacao).dt.tz_localize(None)
d_t.to_excel("dados\tabela_duplicados.xlsx")

# Contagem dos valores que possuem um subscription_id

## Pegando a quantidade de alunos, manipulando o dataframe

In [ ]:
# vamos já usar como base o df com somente transacoes pagas/nao canceladas
df_assinantes = df_status_pago.copy()

# vamos montar uma lista com os id_subscription com o valor 1992
lista_subscription_id_1992 = list()
for i in df_assinantes[(df_assinantes['valor_brl'] == 1992.0) & (df_assinantes['subscription_id'] != 'None')]['subscription_id']:
    lista_subscription_id_1992.append(i)

# agora arrumamos os 'subscription_id' em transacoes que nao era para ter. estava em valores pagos a vista
df_assinantes['subscription_id'] = df_assinantes['subscription_id'].replace(lista_subscription_id_1992, 'None')

# checando oq comentamos acima
# print("--- Com subscription_id ---")
# print(df_assinantes[(df_assinantes['subscription_id'] != "None")]['valor_brl'].value_counts())
# print("\n--- Sem subscription_id ---")
# print(df_assinantes[(df_assinantes['subscription_id'] == "None")]['valor_brl'].value_counts())


# tem valores que seriam de subscription_id que não possuem um subs_id
df_assinantes[ (df_assinantes['valor_brl'] == 399) ]

## Analisando os outros valores com subscription ID

In [ ]:
df_status_pago[(df_status_pago['subscription_id'] != "None")]['valor_brl'].value_counts()

df[(df['valor_brl']==1992) & (df['subscription_id'] != "None")]['subscription_id']

In [ ]:
# AGORA VER TODOS OS VALORES
print("-- Valores que deveriam ter o subscription ID, mas não possuem -- ")
valores = [200, 242, 399, 139, 79]

for val in valores:
    qtde = sum([v for k,v in df_status_pago[(df_status_pago['valor_brl'] == val)]['subscription_id'].value_counts().items() if k == "None"])
    print("| De R$ {}: {}".format(val,qtde))

In [ ]:
print("Média das parcelas com transacoes com subscription_id == 'None'")

valores = [200, 242, 399, 139, 79]

for val in valores:
    qtde = df_status_pago[(df_status_pago['valor_brl'] == val) & (df_status_pago['subscription_id'] == 'None')]['num_parcelas']
    print("Média de parcelas no valor de R$ {}: {}".format(val, qtde.sum() / len(qtde)))


In [ ]:
df_none_1992 = df_status_pago[(df_status_pago['valor_brl'] == 1992) & (df_status_pago['subscription_id'] == 'None')]['num_parcelas']
df_notNone_1992 = df_status_pago[(df_status_pago['valor_brl'] == 1992) & (df_status_pago['subscription_id'] != 'None')]['num_parcelas']
print("Média de parcelas no valor de R$ 1992 (None): {}".format(df_none_1992.sum() / len(df_none_1992)))
print("Média de parcelas no valor de R$ 1992 (not None): {}".format(df_notNone_1992.sum() / len(df_notNone_1992)))
print("Com none: {}".format(len(df_none_1992)))
print("Sem none: {}".format(len(df_notNone_1992)))

In [ ]:
print("Vezes que o email repete | Número de vezes que essa quantia repete")
for i in Counter(df['email'].value_counts()).items():
    print("Qtde: {:2} | Ocorrencias: {}".format(i[0],i[1]))

In [ ]:
# selecionar só a primeira linha
df.loc[[1],]

# retornar um dataframe com os EMAILS duplicados
#df_email_duplicado = df[(df.duplicated(['email'])) & (df['status'] == 'paid')]
#df_email_duplicado = df[(df.duplicated(['email']))]

# COM VALORES PAGO
df_email_duplicado = df_status_pago[(df_status_pago.duplicated(['email'])) & (df_status_pago['status'] == 'paid')]
#df_email_duplicado = df_status_pago[(df_status_pago.duplicated(['email']))]

# printar
print("Colunas: {}".format(len(df_email_duplicado)))
df_email_duplicado